In [ ]:
!pip3 install transformers

     |████████████████████████████████| 1.4MB 9.1MB/s 
     |████████████████████████████████| 2.9MB 30.1MB/s 
     |████████████████████████████████| 890kB 60.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5c066b4c9a5ee4ffc3e8e4306ee43df340ff67a9300e60265acfd853bdcfda90
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Calculating sentiment mesasures for different models

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import pickle

from transformers import pipeline

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Use Huggingface pipeline to estimate sentiment

In [ ]:
nlp = pipeline('sentiment-analysis', device=0)

In [ ]:
news = pickle.load(open('/content/drive/MyDrive/abnormal-distribution-project-data/lean/master.p','rb'))


In [ ]:
sentiment_list = []

for t in tqdm(news.text.to_list()):
    
    if len(nlp.tokenizer(t)['input_ids']) < 512:
            
        sentiment_list.append(nlp(t))
        
    else:
        
        sentiment_list.append('error')


In [ ]:
sentiment_list[0:10]

[[{'label': 'NEGATIVE', 'score': 0.9952332973480225}],
 [{'label': 'POSITIVE', 'score': 0.9998228549957275}],
 [{'label': 'NEGATIVE', 'score': 0.8570035099983215}],
 [{'label': 'POSITIVE', 'score': 0.9972251653671265}],
 [{'label': 'NEGATIVE', 'score': 0.848721981048584}],
 [{'label': 'NEGATIVE', 'score': 0.9951369762420654}],
 [{'label': 'POSITIVE', 'score': 0.9970508813858032}],
 [{'label': 'NEGATIVE', 'score': 0.9991442561149597}],
 [{'label': 'NEGATIVE', 'score': 0.9685935974121094}],
 [{'label': 'NEGATIVE', 'score': 0.9952538013458252}]]

In [ ]:
vec = []
for s in sentiment_list:
  if s == 'error':
    vec.append(np.nan)
  elif s[0]['label'] == 'NEGATIVE':
    vec.append(0)
  else:
    vec.append(1)

nn = news[news.source=='finnhub'][['id','ticker']]
nn = nn.reset_index(drop=True)
nn['score'] = vec
nn.to_csv(SENT_BERT_FINNHUB,index=False)

## Aggregate sentiment into pandas tagged with id, date, ticker

In [ ]:
SENT_FINBERT_TIINGO = '/content/drive/MyDrive/abnormal-distribution-project-data/news_reduced/finbert_tiingo_with_sentiment.csv'
SENT_BERT_TIINGO = '/content/drive/MyDrive/abnormal-distribution-project-data/news_reduced/tiingo_with_sentiment.csv'
SENT_FINBERT_FINNHUB = '/content/drive/MyDrive/abnormal-distribution-project-data/news_reduced/finbert_finnhub_with_sentiment.csv'
SENT_BERT_FINNHUB = '/content/drive/MyDrive/abnormal-distribution-project-data/news_reduced/finnhub_with_sentiment.csv'

In [ ]:
tiingo_finbert_df = pd.read_csv(SENT_FINBERT_TIINGO)
tiingo_bert_df = pd.read_csv(SENT_BERT_TIINGO)
finnhub_finbert_df = pd.read_csv(SENT_FINBERT_FINNHUB)
finnhub_bert_df = pd.read_csv(SENT_BERT_FINNHUB)

In [ ]:
tiingo_finbert_df.head()

,id,date,ticker,label,score
0,26108102,1996-12-31T05:00:00+00:00,msft,LABEL_1,0.999990
1,26108212,1996-12-31T05:00:00+00:00,amzn,LABEL_1,0.999987
2,26108278,1996-12-31T05:00:00+00:00,man,LABEL_1,0.999988
3,26108333,1996-12-31T05:00:00+00:00,msi,LABEL_1,0.999991
4,26107784,1997-02-28T05:00:00+00:00,csco,LABEL_1,0.999991


In [ ]:
tiingo_finbert_df.loc[tiingo_finbert_df.label=='LABEL_0', 'score'] = - tiingo_finbert_df.loc[tiingo_finbert_df.label=='LABEL_0', 'score']
tiingo_finbert_df.loc[tiingo_finbert_df.label=='LABEL_1', 'score'] = 0
#tiingo_finbert_df.loc[tiingo_finbert_df.label=='LABEL_2', 'score'] = 1
tiingo_finbert_df = tiingo_finbert_df[['id', 'ticker', 'score']]
tiingo_finbert_df['source'] = 'tiingo'

In [ ]:
tiingo_finbert_df.head()

,id,ticker,score,source
0,26108102,msft,0.0,tiingo
1,26108212,amzn,0.0,tiingo
2,26108278,man,0.0,tiingo
3,26108333,msi,0.0,tiingo
4,26107784,csco,0.0,tiingo


In [ ]:
news = pickle.load(open('/content/drive/MyDrive/abnormal-distribution-project-data/news_reduced/finnhub_sp1500.p','rb'))
finnhub_finbert_df['id'] = news.reset_index().id
finnhub_finbert_df.loc[finnhub_finbert_df.label=='LABEL_0', 'score'] = - finnhub_finbert_df.loc[finnhub_finbert_df.label=='LABEL_0', 'score']
finnhub_finbert_df.loc[finnhub_finbert_df.label=='LABEL_1', 'score'] = 0
#finnhub_finbert_df.loc[finnhub_finbert_df.label=='LABEL_2', 'score'] = 1
finnhub_finbert_df = finnhub_finbert_df[['id', 'ticker', 'score']]
finnhub_finbert_df.ticker = finnhub_finbert_df.ticker.str.lower()
finnhub_finbert_df['source'] = 'finnhub'

In [ ]:
finnhub_finbert_df.head()

,id,ticker,score,source
0,31438683,a,0.0,finnhub
1,31438682,a,0.0,finnhub
2,31438680,a,0.0,finnhub
3,31438681,a,0.0,finnhub
4,31438679,a,0.0,finnhub


In [ ]:
tiingo_bert_df.loc[tiingo_bert_df.label == 'NEGATIVE','score'] = - tiingo_bert_df.loc[tiingo_bert_df.label == 'NEGATIVE','score']
#tiingo_bert_df.loc[tiingo_bert_df.label == 'POSITIVE','score'] = 1
tiingo_bert_df = tiingo_bert_df.rename(columns={'tickers':'ticker'})
tiingo_bert_df = tiingo_bert_df[['id', 'ticker', 'score']]
tiingo_bert_df['source'] = 'tiingo'

In [ ]:
tiingo_bert_df.head()

,id,ticker,score,source
0,26108102,msft,0.0,tiingo
1,26108212,amzn,1.0,tiingo
2,26108333,msi,0.0,tiingo
3,26107784,csco,1.0,tiingo
4,26107785,intu,0.0,tiingo


In [ ]:
finnhub_bert_df['source'] = 'finnhub'
finnhub_bert_df.head()

,id,ticker,score,source
0,8039795,nyt,0.0,finnhub
1,8039707,hei,1.0,finnhub
2,9093075,msft,0.0,finnhub
3,61246959,bma,1.0,finnhub
4,15811106,f,0.0,finnhub


In [ ]:
MASTER_PATH = '/content/drive/MyDrive/abnormal-distribution-project-data/lean/master.p'
with open(MASTER_PATH,'rb') as pkl_file:
  news_df = pickle.load(pkl_file)

In [ ]:
news_df.head()

,id,text,date,ticker,source,dict_score,_id,open_date,close_date,ret_close_2,ret_close_3,res_close_2,res_close_3,ret_open_2,ret_open_3,dict_ret_close,dict_ret_open,dict_res_close
0,403021,Skyworks Solutions and SanDisk are also seen a...,2010-12-30 00:01:00-05:00,swks,tiingo,1.0,403021,2010-12-30,2010-12-30,1,2,0,0,0,1,2,0,0
1,403083,The insurer's results have been decent but the...,2010-12-30 00:01:00-05:00,all,tiingo,1.0,403083,2010-12-30,2010-12-30,0,0,1,1,0,1,0,0,2
2,403018,The chip maker is expanding into a variety of ...,2010-12-30 00:01:00-05:00,jbl,tiingo,0.0,403018,2010-12-30,2010-12-30,0,0,1,1,0,0,1,1,1
3,403017,The maker of radiation-based products will fet...,2010-12-30 00:01:00-05:00,var,tiingo,0.0,403017,2010-12-30,2010-12-30,0,0,0,1,0,1,1,1,1
4,20396901,Video chat on the iPhone 4 just got a lot more...,2010-12-30 02:39:28-05:00,t,tiingo,0.0,20396901,2010-12-30,2010-12-30,1,1,0,1,0,1,1,1,1


In [ ]:
scores = news_df[['_id','id','date','ticker','source','open_date','close_date','dict_score']]

In [ ]:
scores['bert_baseline'] = np.nan
scores['finbert_baseline'] = np.nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
mer = scores.reset_index().merge(finnhub_bert_df, on=['id','source'])[['index','score']].set_index('index')
scores.loc[mer.index,'bert_baseline'] = mer.score

mer = scores.reset_index().merge(tiingo_bert_df, on=['id','source'])[['index','score']].set_index('index')
scores.loc[mer.index,'bert_baseline'] = mer.score

mer = scores.reset_index().merge(finnhub_finbert_df, on=['id','source'])[['index','score']].set_index('index')
scores.loc[mer.index,'finbert_baseline'] = mer.score

mer = scores.reset_index().merge(tiingo_finbert_df, on=['id','source'])[['index','score']].set_index('index')
scores.loc[mer.index,'finbert_baseline'] = mer.score

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
scores.to_csv('/content/drive/MyDrive/abnormal-distribution-project-data/lean/scores_master.csv', index=False)